## Conversion to fif: SLCH006

This script takes the raw, original data file we recieved from WashU and converts it to a fif file with the photodiode as a new stim channel


Just need to change fields in `prep_paths`

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import signal, stats
import mat73
import re
from neurodsp.timefrequency import compute_wavelet_transform
from BCI2kReader import BCI2kReader as b2k
import os
import mne
from tabulate import tabulate
import IPython
import seaborn as sns

In [10]:
## prep paths

subject = 'SLCH006'
orig_data_fi = '/home/brooke/knight_server/remote/WashU/data/PacmanTask/SLCH006/Pacman/ECOG001/ECOGS001R01.dat'
raw_data_dir = f"/home/brooke/pacman/raw_data/{subject}/ieeg"

In [3]:
## load data

with b2k.BCI2kReader(orig_data_fi) as data: #opens a stream to the dat file
    my_signals = data.signals
    my_states = data.states
    my_params = data.parameters

In [4]:
## check structure if necessary

# my_states
# my_params

{'SourceCh': 256,
 'SampleBlockSize': 200,
 'SamplingRate': 2000,
 'ChannelNames': ['A1',
  'A2',
  'A3',
  'A4',
  'REF1',
  'REF2',
  'A5',
  'A6',
  'A7',
  'A8',
  'A9',
  'A10',
  'A11',
  'A12',
  'A13',
  'A14',
  'B1',
  'B2',
  'B3',
  'B4',
  'B5',
  'B6',
  'B7',
  'B8',
  'B9',
  'B10',
  'B11',
  'B12',
  'B13',
  'Fp1',
  'C1',
  'C2',
  'C3',
  'C4',
  'C5',
  'C6',
  'C7',
  'C8',
  'C9',
  'C10',
  'C11',
  'C12',
  'C13',
  'Fp2',
  'D1',
  'D2',
  'D3',
  'D4',
  'D5',
  'D6',
  'D7',
  'D8',
  'D9',
  'D10',
  'D11',
  'D12',
  'E1',
  'E2',
  'E3',
  'E4',
  'E5',
  'E6',
  'E7',
  'E8',
  'E9',
  'E10',
  'E11',
  'E12',
  'F1',
  'F2',
  'F3',
  'F4',
  'F5',
  'F6',
  'F7',
  'F8',
  'F9',
  'F10',
  'F11',
  'F12',
  'G1',
  'G2',
  'G3',
  'G4',
  'G5',
  'G6',
  'G7',
  'G8',
  'G9',
  'G10',
  'G11',
  'G12',
  'H1',
  'H2',
  'H3',
  'H4',
  'H5',
  'H6',
  'H7',
  'H8',
  'H9',
  'H10',
  'H11',
  'H12',
  'i1',
  'i2',
  'i3',
  'i4',
  'i5',
  'i6',
  'i

In [5]:
## create info field 

info = mne.create_info(ch_names = my_params['ChannelNames'],
                       sfreq = my_params['SamplingRate'],
                       ch_types = 'seeg')

info

<Info | 7 non-empty values
 bads: []
 ch_names: A1, A2, A3, A4, REF1, REF2, A5, A6, A7, A8, A9, A10, A11, A12, ...
 chs: 256 sEEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 1000.0 Hz
 meas_date: unspecified
 nchan: 256
 projs: []
 sfreq: 2000.0 Hz
>

In [6]:
## create mne object
mne_df = mne.io.RawArray(my_signals, info)

Creating RawArray with float64 data, n_channels=256, n_times=3201200
    Range : 0 ... 3201199 =      0.000 ...  1600.600 secs
Ready.


In [7]:
## create stim channel and add it to mne object

info = mne.create_info(['STI'], my_params['SamplingRate'], ['stim'])
stim_raw = mne.io.RawArray( my_states['Trial_on_off'], info)
mne_df.add_channels([stim_raw], force_update_info=True)

Creating RawArray with float64 data, n_channels=1, n_times=3201200
    Range : 0 ... 3201199 =      0.000 ...  1600.600 secs
Ready.


<RawArray | 257 x 3201200 (1600.6 s), ~6.13 GB, data loaded>

In [11]:
## save it

mne_df.save(f"{raw_data_dir}/{subject}_raw_ieeg.fif", overwrite = False)

Writing /home/brooke/pacman/raw_data/SLCH006/ieeg/SLCH006_raw_ieeg.fif
Overwriting existing file.
Writing /home/brooke/pacman/raw_data/SLCH006/ieeg/SLCH006_raw_ieeg-1.fif
Closing /home/brooke/pacman/raw_data/SLCH006/ieeg/SLCH006_raw_ieeg-1.fif
Closing /home/brooke/pacman/raw_data/SLCH006/ieeg/SLCH006_raw_ieeg.fif
[done]
